En esta práctica de laboratorio, realizaremos un análisis de datos exploratorios (EDA) para encontrar algunos patrones en los datos y determinar cuál sería la etiqueta para entrenar modelos supervisados.

En el conjunto de datos, hay varios casos diferentes en los que el propulsor no aterrizó con éxito. A veces se intentó un aterrizaje pero fracasó debido a un accidente; por ejemplo, True Ocean significa que el resultado de la misión aterrizó con éxito en una región específica del océano, mientras que False Ocean significa que el resultado de la misión no aterrizó sin éxito en una región específica del océano. RTLS verdadero significa que el resultado de la misión aterrizó exitosamente en una plataforma terrestre. RTLS falso significa que el resultado de la misión no aterrizó exitosamente en una plataforma terrestre. ASDS verdadero significa que el resultado de la misión aterrizó exitosamente en un barco no tripulado. ASDS falso significa que el resultado de la misión no aterrizó exitosamente en un barco no tripulado.

En esta práctica de laboratorio convertiremos principalmente esos resultados en etiquetas de entrenamiento, donde 1 significa que el refuerzo aterrizó exitosamente y 0 significa que no tuvo éxito.

Objetivos\
Realizar análisis de datos exploratorios y determinar etiquetas de capacitación

Análisis exploratorio de datos
Determinar etiquetas de entrenamiento

In [2]:
import pandas as pd
import numpy as np

In [3]:
df=pd.read_csv("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/dataset_part_1.csv")
df.head(10)

,FlightNumber,Date,BoosterVersion,PayloadMass,Orbit,LaunchSite,Outcome,Flights,GridFins,Reused,Legs,LandingPad,Block,ReusedCount,Serial,Longitude,Latitude
0,1,2010-06-04,Falcon 9,6104.959412,LEO,CCAFS SLC 40,None None,1,False,False,False,NaN,1.0,0,B0003,-80.577366,28.561857
1,2,2012-05-22,Falcon 9,525.000000,LEO,CCAFS SLC 40,None None,1,False,False,False,NaN,1.0,0,B0005,-80.577366,28.561857
2,3,2013-03-01,Falcon 9,677.000000,ISS,CCAFS SLC 40,None None,1,False,False,False,NaN,1.0,0,B0007,-80.577366,28.561857
3,4,2013-09-29,Falcon 9,500.000000,PO,VAFB SLC 4E,False Ocean,1,False,False,False,NaN,1.0,0,B1003,-120.610829,34.632093
4,5,2013-12-03,Falcon 9,3170.000000,GTO,CCAFS SLC 40,None None,1,False,False,False,NaN,1.0,0,B1004,-80.577366,28.561857
5,6,2014-01-06,Falcon 9,3325.000000,GTO,CCAFS SLC 40,None None,1,False,False,False,NaN,1.0,0,B1005,-80.577366,28.561857
6,7,2014-04-18,Falcon 9,2296.000000,ISS,CCAFS SLC 40,True Ocean,1,False,False,True,NaN,1.0,0,B1006,-80.577366,28.561857
7,8,2014-07-14,Falcon 9,1316.000000,LEO,CCAFS SLC 40,True Ocean,1,False,False,True,NaN,1.0,0,B1007,-80.577366,28.561857
8,9,2014-08-05,Falcon 9,4535.000000,GTO,CCAFS SLC 40,None None,1,False,False,False,NaN,1.0,0,B1008,-80.577366,28.561857
9,10,2014-09-07,Falcon 9,4428.000000,GTO,CCAFS SLC 40,None None,1,False,False,False,NaN,1.0,0,B1011,-80.577366,28.561857


### Identificar y calcular el porcentaje de los valores faltantes en cada atributo.

In [4]:
df.isnull().sum()/len(df)*100

FlightNumber       0.000000
Date               0.000000
BoosterVersion     0.000000
PayloadMass        0.000000
Orbit              0.000000
LaunchSite         0.000000
Outcome            0.000000
Flights            0.000000
GridFins           0.000000
Reused             0.000000
Legs               0.000000
LandingPad        28.888889
Block              0.000000
ReusedCount        0.000000
Serial             0.000000
Longitude          0.000000
Latitude           0.000000
dtype: float64

In [5]:
df.dtypes

FlightNumber        int64
Date               object
BoosterVersion     object
PayloadMass       float64
Orbit              object
LaunchSite         object
Outcome            object
Flights             int64
GridFins             bool
Reused               bool
Legs                 bool
LandingPad         object
Block             float64
ReusedCount         int64
Serial             object
Longitude         float64
Latitude          float64
dtype: object

## TAREA 1: Calcular el número de lanzamientos en cada sitio
Los datos contienen varias instalaciones de lanzamiento de Space X: Complejo de lanzamiento espacial de Cabo Cañaveral 40 VAFB SLC 4E, Complejo de lanzamiento espacial de la Base de la Fuerza Aérea Vandenberg 4E (SLC-4E), Complejo de lanzamiento del Centro espacial Kennedy 39A KSC LC 39A. Se coloca la ubicación de cada lanzamiento. en la columna Sitio de lanzamiento

## A continuación, veamos la cantidad de lanzamientos de cada sitio.

Utilice el método value_counts() en la columna LaunchSite para determinar el número de lanzamientos en cada sitio:

In [6]:
conteo_lanzamientos = df["LaunchSite"].value_counts()
print(conteo_lanzamientos)

LaunchSite
CCAFS SLC 40    55
KSC LC 39A      22
VAFB SLC 4E     13
Name: count, dtype: int64


### Cada lanzamiento apunta a una órbita dedicada y estos son algunos tipos de órbitas comunes:

LEO: La órbita terrestre baja (LEO) es una órbita centrada en la Tierra con una altitud de 2000 km (1200 millas) o menos (aproximadamente un tercio del radio de la Tierra),[1] o con al menos 11,25 períodos por día ( un período orbital de 128 minutos o menos) y una excentricidad inferior a 0,25.[2] La mayoría de los objetos creados por el hombre en el espacio exterior se encuentran en LEO [1].

VLEO: Las órbitas terrestres muy bajas (VLEO) se pueden definir como órbitas con una altitud media inferior a 450 km. Operar en estas órbitas puede proporcionar una serie de beneficios a las naves espaciales de observación de la Tierra, ya que operan más cerca del lugar de observación[2].

GTO Una órbita geosincrónica es una órbita terrestre alta que permite a los satélites igualar la rotación de la Tierra. Situada a 35.786 kilómetros (22.236 millas) sobre el ecuador de la Tierra, esta posición es un lugar valioso para monitorear el clima, las comunicaciones y la vigilancia. Debido a que el satélite orbita a la misma velocidad que gira la Tierra, el satélite parece permanecer en su lugar en una sola longitud, aunque puede desplazarse de norte a sur”, escribió la NASA en su sitio web del Observatorio de la Tierra [3].

SSO (o SO): Es una órbita sincrónica con el Sol, también llamada órbita heliosincrónica, es una órbita casi polar alrededor de un planeta, en la que el satélite pasa sobre cualquier punto determinado de la superficie del planeta a la misma hora solar media local [4] .

ES-L1: En los puntos de Lagrange las fuerzas gravitacionales de los dos cuerpos grandes se anulan de tal manera que un objeto pequeño colocado en órbita allí está en equilibrio con respecto al centro de masa de los cuerpos grandes. L1 es uno de esos puntos entre el sol y la tierra [5].

HEO Una órbita altamente elíptica, es una órbita elíptica con alta excentricidad, generalmente refiriéndose a una alrededor de la Tierra [6].

ISS Una estación espacial modular (satélite artificial habitable) en órbita terrestre baja. Es un proyecto multinacional de colaboración entre cinco agencias espaciales participantes: NASA (Estados Unidos), Roscosmos (Rusia), JAXA (Japón), ESA (Europa) y CSA (Canadá) [7]

MEO Órbitas geocéntricas que varían en altitud desde 2000 km (1200 millas) hasta justo debajo de la órbita geosincrónica a 35,786 kilómetros (22,236 millas). También conocida como órbita circular intermedia. Estos están "más comúnmente a 20.200 kilómetros (12.600 millas), o 20.650 kilómetros (12.830 millas), con un período orbital de 12 horas [8]

Órbitas geocéntricas HEO por encima de la altitud de la órbita geosincrónica (35.786 km o 22.236 millas) [9]

GEO Es una órbita geosincrónica circular a 35.786 kilómetros (22.236 millas) sobre el ecuador terrestre y siguiendo la dirección de rotación de la Tierra [10]

PO Es un tipo de satélites en los que un satélite pasa por encima o casi por encima de ambos polos del cuerpo que se orbita (normalmente un planeta como la Tierra [11]

algunos se muestran en el siguiente gráfico:

![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/api/Images/Orbits.png)

## TAREA 2: Calcular el número y la aparición de cada órbita.

Utilice el método .value_counts() para determinar el número y la aparición de cada órbita en la columna Orbit.

In [7]:
conteo_orbitas = df["Orbit"].value_counts()
print(conteo_orbitas)

Orbit
GTO      27
ISS      21
VLEO     14
PO        9
LEO       7
SSO       5
MEO       3
ES-L1     1
HEO       1
SO        1
GEO       1
Name: count, dtype: int64


## TAREA 3: Calcular el número y la ocurrencia del resultado de la misión de las órbitas¶
Utilice el método .value_counts() en la columna Outcome para determinar el número de landing_outcomes. Luego asígnelo a una variable landing_outcomes.

In [10]:
landing_outcomes = df["Outcome"].value_counts()
print(landing_outcomes)

Outcome
True ASDS      41
None None      19
True RTLS      14
False ASDS      6
True Ocean      5
False Ocean     2
None ASDS       2
False RTLS      1
Name: count, dtype: int64


True Ocean significa que el resultado de la misión aterrizó con éxito en una región específica del océano, mientras que False Ocean significa que el resultado de la misión aterrizó sin éxito en una región específica del océano. RTLS true significa que el resultado de la misión aterrizó exitosamente en una plataforma terrestre. RTLS false significa que el resultado de la misión no aterrizó exitosamente en una plataforma terrestre. ASDS true significa que el resultado de la misión aterrizó exitosamente en un barco no tripulado. ASDS false significa que el resultado de la misión no aterrizó exitosamente en un barco no tripulado. None ASDS y None None, estos representan un fracaso en el aterrizaje.

In [11]:
for i, outcome in enumerate(landing_outcomes.keys()):
    print(i,outcome)

0 True ASDS
1 None None
2 True RTLS
3 False ASDS
4 True Ocean
5 False Ocean
6 None ASDS
7 False RTLS


Creamos un conjunto de resultados donde la segunda etapa no aterrizó con éxito:

In [13]:
bad_outcomes=set(landing_outcomes.keys()[[1,3,5,6,7]])
bad_outcomes

{'False ASDS', 'False Ocean', 'False RTLS', 'None ASDS', 'None None'}

## TAREA 4: Crear una etiqueta de resultado de aterrizaje a partir de la columna Resultado¶
Usando el Outcome, cree una lista donde el elemento sea cero si la fila correspondiente en Outcome está en el conjunto bad_outcome; de lo contrario, es uno. Luego asígnalo a la variable landing_class:

In [16]:
landing_class = [0 if Outcome in bad_outcomes else 1 for Outcome in df ['Outcome']  ]

Esta variable representará la variable de clasificación que representa el resultado de cada lanzamiento. Si el valor es cero, la primera etapa no aterrizó exitosamente; uno significa que la primera etapa aterrizó con éxito.

In [17]:
df['Class']=landing_class
df[['Class']].head(8)

,Class
0,0
1,0
2,0
3,0
4,0
5,0
6,1
7,1


In [18]:
df.head(5)

,FlightNumber,Date,BoosterVersion,PayloadMass,Orbit,LaunchSite,Outcome,Flights,GridFins,Reused,Legs,LandingPad,Block,ReusedCount,Serial,Longitude,Latitude,Class
0,1,2010-06-04,Falcon 9,6104.959412,LEO,CCAFS SLC 40,None None,1,False,False,False,NaN,1.0,0,B0003,-80.577366,28.561857,0
1,2,2012-05-22,Falcon 9,525.000000,LEO,CCAFS SLC 40,None None,1,False,False,False,NaN,1.0,0,B0005,-80.577366,28.561857,0
2,3,2013-03-01,Falcon 9,677.000000,ISS,CCAFS SLC 40,None None,1,False,False,False,NaN,1.0,0,B0007,-80.577366,28.561857,0
3,4,2013-09-29,Falcon 9,500.000000,PO,VAFB SLC 4E,False Ocean,1,False,False,False,NaN,1.0,0,B1003,-120.610829,34.632093,0
4,5,2013-12-03,Falcon 9,3170.000000,GTO,CCAFS SLC 40,None None,1,False,False,False,NaN,1.0,0,B1004,-80.577366,28.561857,0


### Podemos utilizar la siguiente línea de código para determinar la tasa de éxito:

In [19]:
df["Class"].mean()

0.6666666666666666

Ahora podemos exportarlo a un CSV para la siguiente sección, pero para que las respuestas sean consistentes, en la próxima práctica de laboratorio proporcionaremos datos en un rango de fechas preseleccionado.

In [20]:
df.to_csv("dataset_part_2.csv", index=False)